# Change Reset Pattern

In [1]:
import decimal

In [2]:
decimal.getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [5]:
decimal.getcontext().prec = 14

In [6]:
decimal.getcontext()

Context(prec=14, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

In [7]:
decimal.getcontext().prec = 28

In [14]:
old_prec = decimal.getcontext().prec
decimal.getcontext().prec = 4
print(decimal.Decimal(1) / decimal.Decimal(3))
decimal.getcontext().prec = old_prec
print(decimal.Decimal(1) / decimal.Decimal(3))

0.3333
0.3333333333333333333333333333


In [15]:
class Precision:
    def __init__(self, prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec
        
    def __enter__(self):
        decimal.getcontext().prec = self.prec
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        decimal.getcontext().prec = self.current_prec
        return False

In [17]:
print("prec before", decimal.getcontext().prec)
with Precision(2):
    print("prec in", decimal.getcontext().prec)
    print(decimal.Decimal(1) / decimal.Decimal(3))
print("prec after", decimal.getcontext().prec)

prec before 28
prec in 2
0.33
prec after 28


In [21]:
print(decimal.getcontext().prec)
with decimal.localcontext() as ctx:
    ctx.prec = 10
    print(decimal.getcontext().prec)
print(decimal.getcontext().prec)

28
10
28


# Start Stop Pattern

In [22]:
from time import perf_counter, sleep

In [31]:
class Timer:
    def __init__(self):
        self.elapsed = 0
        self.start = None
        self.stop = None
        
    def __enter__(self):
        self.start = perf_counter()
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False

In [29]:
with Timer() as timer:
    sleep(1)

In [32]:
timer.start, timer.stop, timer.elapsed

(53837.672258083, 53838.677282666, 1.0050245829988853)

In [33]:
import sys

In [34]:
class OutToFile:
    def __init__(self, fname):
        self.fname = fname
        self._current_stdout = sys.stdout
        
    def __enter__(self):
        self._file = open(self.fname, "w")
        sys.stdout = self._file
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        self._file.close()
        sys.stdout = self._current_stdout

In [41]:
with OutToFile("test.txt"):
    print(sys.stdout)
    print("Line 1")
    print("Line 2")

In [36]:
print("Line 1")

Line 1


In [37]:
sys.stdout

In [42]:
open("test.txt", "r").read()

"<_io.TextIOWrapper name='test.txt' mode='w' encoding='UTF-8'>\nLine 1\nLine 2\n"

In [43]:
class Tag:
    def __init__(self, tag):
        self.tag = tag
        
    def __enter__(self):
        print(f"<{self.tag}>", end="")
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"</{self.tag}>", end="")

In [46]:
with OutToFile("test.txt"):
    with Tag("p"):
        print("some", end="")
        with Tag("b"):
            print("bold", end="")
        print("text", end="")

In [69]:
class ListMaker:
    def __init__(self, indent=4):
        self._indent = indent
        self._current_indent = 0 
    
    def __enter__(self):
        self._current_indent += self._indent
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self._current_indent -= self._indent
        return False
    
    def print(self, arg):
        print(f"{' ' * self._current_indent}{arg}")

In [71]:
with ListMaker(indent=4) as lm:
    lm.print("lm0")
    with lm:
        lm.print("lol")
        

    lm0
        lol
